In [1]:
!pip install transformers

In [2]:
!pip install dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 23.9 MB/s eta 0:00:00


In [3]:
!pip install soundfile
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 56.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 50.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 28.3 MB/s eta 0:00:0000:0100:01
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23702 sha256=e19abbb944499a548a0d59276336907582a20c0e766bed7aa609e45e792e6fcc
  Stored in directory: /root/.cache/pip/wheels/7a/6a/b0/92760a6d6bc2bff5464970af910c0b0b921390993f3199cdf7
Successfully built audioread


In [4]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wzlj8okt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-wzlj8okt
  Resolved https://github.com/huggingface/transformers to commit 5bb4430edc7df9f9950d412d98bbe505cc4d328b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7352645 sha256=cac01cc781016edc428e7060ed183a2bcfa18785487072be3edb16b2817da046
  Stored in directory: /tmp/pip-ephem-wheel-cache-idp19hc4/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing 

In [5]:
!pip install gradio
!pip install -U typing_extensions
!pip install -U pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 46.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 1

In [6]:
!pip install --upgrade pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.7/390.7 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.2/217.2 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
from huggingface_hub import notebook_login

notebook_login()

## Speech to Speech Translation

### Speech Translation

In [1]:
import torch
from transformers import pipeline
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline("automatic-speech-recognition", model="ptah23/whisper-small-af-ZA", device=device)

In [2]:
device

'cuda:0'

In [3]:
from datasets import load_dataset
dataset=load_dataset("google/fleurs", "af_za", split="validation", streaming=True)
iterator = iter(dataset)
sample = next(iterator)

In [4]:
import IPython
IPython.display.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [5]:
def translate(audio):
    outputs = pipe(audio, max_new_tokens=256, generate_kwargs={"task":"translate"})
    return outputs["text"]

In [6]:
translate(sample["audio"].copy())

" in 1989 he helped for brooks and groening to skip the simpsons and he was responsible for the first writer's span for that program"

In [7]:
sample = next(iterator)
IPython.display.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [8]:
translate(sample["audio"].copy())

'hulle is cooler as die omligende omgewing in die dag en warmer in die nag'

### Text-to-speech

In [9]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [10]:
model.to(device)
vocoder.to(device)

SpeechT5HifiGan(
  (conv_pre): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
  (upsampler): ModuleList(
    (0): ConvTranspose1d(512, 256, kernel_size=(8,), stride=(4,), padding=(2,))
    (1): ConvTranspose1d(256, 128, kernel_size=(8,), stride=(4,), padding=(2,))
    (2): ConvTranspose1d(128, 64, kernel_size=(8,), stride=(4,), padding=(2,))
    (3): ConvTranspose1d(64, 32, kernel_size=(8,), stride=(4,), padding=(2,))
  )
  (resblocks): ModuleList(
    (0): HifiGanResidualBlock(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
        (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
      )
      (convs2): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (2): Conv1d(256, 256, ker

In [11]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Reusing dataset cmu-arctic-xvectors (/root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f)


In [12]:
def synthesize(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(inputs["input_ids"].to(device), speaker_embeddings.to(device), vocoder=vocoder)
    return speech.cpu()

In [13]:
speech = synthesize("He's a dude, I'm a dude, she's a dude. We're all dudes yeah!")
IPython.display.Audio(speech, rate=16_000)

### Create a STST demo

In [14]:
import numpy as np
target_dtype = np.int16
max_range = np.iinfo(target_dtype).max

def speech_to_speech_translation(audio):
    translated_text = translate(audio)
    synthesized_speech = synthesize(translated_text)
    synthesized_speech = (synthesized_speech.numpy() * max_range).astype(np.int16)
    return 16000, synthesized_speech

sampling_rate, synthesized_speech = speech_to_speech_translation(sample["audio"])
IPython.display.Audio(synthesized_speech, rate=sampling_rate)

In [ ]:
import gradio as gr

demo = gr.Blocks()

mic_translate = gr.Interface(fn=speech_to_speech_translation,
                             inputs=gr.Audio(source="microphone", type="filepath"),
                             outputs=gr.Audio(label="Generated Speech", type="numpy"),
                            )
file_translate = gr.Interface(fn=speech_to_speech_translation,
                              inputs=gr.Audio(souce="upload", type="filepath"),
                              outputs=gr.Audio(label="Generated Speech", type="numpy"),
                             )
                                              
with demo:
    gr.TabbedInterface([mic_translate, file_translate], ["Microphone", "Audio File"])
    
demo.launch(debug=True, share=True) 

/tmp/ipykernel_1291/3509222265.py:10: GradioUnusedKwargWarning: You have unused kwarg parameters in Audio, please remove them: {'souce': 'upload'}
  inputs=gr.Audio(souce="upload", type="filepath"),
/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:934: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e70a703eb48e12690a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Creating a voice assistant

### Wake word detection

In [7]:
from transformers import pipeline
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

classifier = pipeline("audio-classification", model="MIT/ast-finetuned-speech-commands-v2", device=device)

In [8]:
classifier.model.config.id2label

{0: 'backward',
 1: 'follow',
 2: 'five',
 3: 'bed',
 4: 'zero',
 5: 'on',
 6: 'learn',
 7: 'two',
 8: 'house',
 9: 'tree',
 10: 'dog',
 11: 'stop',
 12: 'seven',
 13: 'eight',
 14: 'down',
 15: 'six',
 16: 'forward',
 17: 'cat',
 18: 'right',
 19: 'visual',
 20: 'four',
 21: 'wow',
 22: 'no',
 23: 'nine',
 24: 'off',
 25: 'three',
 26: 'left',
 27: 'marvin',
 28: 'yes',
 29: 'up',
 30: 'sheila',
 31: 'happy',
 32: 'bird',
 33: 'go',
 34: 'one'}

In [10]:
classifier.model.config.id2label[27]

'marvin'

In [12]:
from transformers.pipelines.audio_utils import ffmpeg_microphone_live

def launch_fn(wake_word="marvin",
              prob_threshold=0.5,
              chunk_length_s=2.0,
              stream_chunk_s=0.25,
              debug=False,
             ):
    
    if wake_word not in classifier.model.config.label2id.keys():
        raise ValueError(
            f"Wake word {wake_word} not in set of valid class labels, pick a wake word in the set {classifier.model.config.label2id.keys()}.")
    sampling_rate = classifier.feature_extractor.sampling_rate
    mic = ffmpeg_microphone_live(sampling_rate=sampling_rate, chunk_length_s=chunk_length_s, stream_chunk_s=stream_chunk_s)
    print("Listening for wake word...")
    for prediction in classifier(mic):
        prediction = prediction[0]
        if debug:
            print(prediction)
        if prediction["label"] == wake_word:
            if prediction["score"] > prob_threshold:
                return True
            
            

In [15]:
launch_fn(debug=True)

Listening for wake word...


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


## Transcribe a meeting

### Speaker diarization

In [8]:
# first agree to terms on https://huggingface.co/pyannote/segmentation
# and https://huggingface.co/pyannote/speaker-diarization
from pyannote.audio import Pipeline

diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token=True)

In [9]:
from datasets import load_dataset
concatenated_librispeech = load_dataset("sanchit-gandhi/concatenated_librispeech", split="train", streaming=True)
sample = next(iter(concatenated_librispeech))

Using custom data configuration sanchit-gandhi--concatenated_librispeech-6011bc7607bb9cdb


In [ ]:
sample

In [28]:
import IPython
IPython.display.Audio(sample["audio"]["bytes"], rate=16_000)

In [14]:
import torchaudio
import numpy as np
import io
input_tensor, sample_rate = torchaudio.load(io.BytesIO(sample["audio"]["bytes"]))

outputs = diarization_pipeline({"waveform":input_tensor, "sample_rate":sample_rate})
outputs.for_json()["content"]

[{'segment': {'start': 0.4978125, 'end': 14.520937500000002},
  'track': 'B',
  'label': 'SPEAKER_01'},
 {'segment': {'start': 15.364687500000002, 'end': 21.3721875},
  'track': 'A',
  'label': 'SPEAKER_00'}]

### Speech transcription

In [15]:
from transformers import pipeline
asr_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-base")

In [21]:
asr_pipeline({"array": input_tensor.numpy()[0], "sampling_rate": 16_000} , generate_kwargs={"max_new_tokens":256}, return_timestamps=True)

{'text': " The second and importance is as follows. Sovereignty may be defined to be the right of making laws. In France, the king really exercises a portion of the sovereign power, since the laws have no weight. He was in a favored state of mind, owing to the blight his wife's action threatened to cast upon his entire future.",
 'chunks': [{'timestamp': (0.0, 3.56),
   'text': ' The second and importance is as follows.'},
  {'timestamp': (3.56, 7.84),
   'text': ' Sovereignty may be defined to be the right of making laws.'},
  {'timestamp': (7.84, 13.88),
   'text': ' In France, the king really exercises a portion of the sovereign power, since the laws have'},
  {'timestamp': (13.88, 15.48), 'text': ' no weight.'},
  {'timestamp': (15.48, 19.44),
   'text': " He was in a favored state of mind, owing to the blight his wife's action threatened to"},
  {'timestamp': (19.44, 21.28), 'text': ' cast upon his entire future.'}]}

### Speechbox

In [22]:
!pip install git+https://github.com/huggingface/speechbox

  Cloning https://github.com/huggingface/speechbox to /tmp/pip-req-build-oiok6vpx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/speechbox /tmp/pip-req-build-oiok6vpx
  Resolved https://github.com/huggingface/speechbox to commit 96d2d1a180252d92263f862a1cd25a48860f1aed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for speechbox: filename=speechbox-0.2.1-py3-none-any.whl size=20302 sha256=c15190beb157d65cc82efdac0838d62e8dfc381d18c866797a11955975df9b95
  Stored in directory: /tmp/pip-ephem-wheel-cache-r15rz6qb/wheels/3f/d0/49/7d5e347d1d2c793a7b4472cd71249c863f7e1dc004698ef2f2
Successfully built speechbox


In [23]:
from speechbox import ASRDiarizationPipeline
pipeline = ASRDiarizationPipeline(asr_pipeline=asr_pipeline, diarization_pipeline=diarization_pipeline)
## or
# pipeline = ASRDiarizationPipeline.from_pretrained("openai/whisper-base")

In [24]:
pipeline({"array": input_tensor.numpy()[0], "sampling_rate": 16_000})

[{'speaker': 'SPEAKER_01',
  'text': ' The second and importance is as follows. Sovereignty may be defined to be the right of making laws. In France, the king really exercises a portion of the sovereign power, since the laws have no weight.',
  'timestamp': (0.0, 15.48)},
 {'speaker': 'SPEAKER_00',
  'text': " He was in a favored state of mind, owing to the blight his wife's action threatened to cast upon his entire future.",
  'timestamp': (15.48, 21.28)}]

In [25]:
def tuple_to_string(start_end_tuple, ndigits=1):
    return str((round(start_end_tuple[0], ndigits), round(start_end_tuple[1], ndigits)))


def format_as_transcription(raw_segments):
    return "\n\n".join(
        [
            chunk["speaker"] + " " + tuple_to_string(chunk["timestamp"]) + chunk["text"]
            for chunk in raw_segments
        ]
    )

In [26]:
outputs = pipeline({"array": input_tensor.numpy()[0], "sampling_rate": 16_000})
format_as_transcription(outputs)

"SPEAKER_01 (0.0, 15.5) The second and importance is as follows. Sovereignty may be defined to be the right of making laws. In France, the king really exercises a portion of the sovereign power, since the laws have no weight.\n\nSPEAKER_00 (15.5, 21.3) He was in a favored state of mind, owing to the blight his wife's action threatened to cast upon his entire future."